# DATA PREPARATION PART 2

In [38]:
import pandas as pd

In [39]:
data = pd.read_csv('../../data/processed/data.csv')

In [40]:
data = (data.drop(columns='Unnamed: 0')
            .sort_values(by=['file_name', 'end_of_period'], ascending=False))

In [41]:
data['total_liabilities'] = data['current_liabilities'] + data['non_current_liabilities']

In [42]:
data.head(3)

,end_of_period,total_assets,non_current_assets,current_assets,property_plant_equipment,intangible_assets,inventories,trade_receivables,cash_and_cash_equivalents,equity_shareholders_of_the_parent,...,current_liabilities,non_current_loans_and_borrowings,financial_liabilities_loans_borrowings,total_shares,file_name,company_name,ticker,sector,target,total_liabilities
7674,2022-07-01,2733389.0,1720912.0,1012477.0,1025678.0,337294.0,171367.0,826650.0,6249.0,304293.0,...,1468424.0,875000.0,85884.0,10271.0,ZYWIEC.xlsx,Grupa Żywiec SA,ZWC,napoje,494.0,2429096.0
7673,2022-04-01,2981239.0,1730362.0,1250877.0,1031440.0,340644.0,202267.0,1039206.0,6404.0,288397.0,...,1631516.0,975000.0,96674.0,10271.0,ZYWIEC.xlsx,Grupa Żywiec SA,ZWC,napoje,510.0,2692842.0
7672,2021-10-01,2508447.0,1726721.0,781726.0,1050251.0,347755.0,140983.0,570261.0,2510.0,216089.0,...,2209321.0,0.0,1094585.0,10271.0,ZYWIEC.xlsx,Grupa Żywiec SA,ZWC,napoje,462.0,2292358.0


In [43]:
attributes = [
    'total_assets', 'non_current_assets', 'current_assets',
    'property_plant_equipment', 'intangible_assets', 'inventories',
    'trade_receivables', 'cash_and_cash_equivalents', 'equity_shareholders_of_the_parent',
    'share_capital', 'retained_earning_accumulated_losses', 'non_current_liabilities',
    'current_liabilities', 'non_current_loans_and_borrowings', 'financial_liabilities_loans_borrowings'
]

def add_change_features(df, attributes):
    for attr in attributes:
        df[f'{attr}_change_1q'] = df.groupby('file_name')[attr].diff(1)
        df[f'{attr}_change_2q'] = df.groupby('file_name')[attr].diff(2)
        df[f'{attr}_change_4q'] = df.groupby('file_name')[attr].diff(4)
        
    return df

df = add_change_features(data, attributes)

In [44]:
df.head(10)

,end_of_period,total_assets,non_current_assets,current_assets,property_plant_equipment,intangible_assets,inventories,trade_receivables,cash_and_cash_equivalents,equity_shareholders_of_the_parent,...,non_current_liabilities_change_4q,current_liabilities_change_1q,current_liabilities_change_2q,current_liabilities_change_4q,non_current_loans_and_borrowings_change_1q,non_current_loans_and_borrowings_change_2q,non_current_loans_and_borrowings_change_4q,financial_liabilities_loans_borrowings_change_1q,financial_liabilities_loans_borrowings_change_2q,financial_liabilities_loans_borrowings_change_4q
7674,2022-07-01,2733389.0,1720912.0,1012477.0,1025678.0,337294.0,171367.0,826650.0,6249.0,304293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7673,2022-04-01,2981239.0,1730362.0,1250877.0,1031440.0,340644.0,202267.0,1039206.0,6404.0,288397.0,...,NaN,163092.0,NaN,NaN,100000.0,NaN,NaN,10790.0,NaN,NaN
7672,2021-10-01,2508447.0,1726721.0,781726.0,1050251.0,347755.0,140983.0,570261.0,2510.0,216089.0,...,NaN,577805.0,740897.0,NaN,-975000.0,-875000.0,NaN,997911.0,1008701.0,NaN
7671,2021-07-01,2649304.0,1796783.0,852521.0,1028957.0,350722.0,145833.0,682491.0,1929.0,395450.0,...,NaN,-20032.0,557773.0,NaN,0.0,-975000.0,NaN,-231463.0,766448.0,NaN
7670,2021-04-01,3037763.0,1901297.0,1136466.0,1031984.0,353845.0,164077.0,952499.0,3925.0,285629.0,...,-394964.0,-2863.0,-22895.0,718002.0,500000.0,500000.0,-375000.0,-170113.0,-401576.0,607125.0
7669,2020-10-01,2513606.0,1729256.0,784350.0,1053811.0,360038.0,109311.0,612962.0,1328.0,177079.0,...,152357.0,-1063582.0,-1066445.0,-508672.0,650000.0,1150000.0,175000.0,-628925.0,-799038.0,-32590.0
7668,2020-07-01,2836562.0,1700588.0,1072876.0,1058326.0,363589.0,177022.0,885413.0,1384.0,266213.0,...,998668.0,361554.0,-702028.0,-724923.0,-150000.0,500000.0,1000000.0,67038.0,-561887.0,-963463.0
7667,2020-04-01,3000499.0,1628960.0,1371539.0,1072992.0,378553.0,177517.0,1138058.0,1969.0,138518.0,...,1141615.0,171403.0,532957.0,-533488.0,150000.0,0.0,1150000.0,47913.0,114951.0,-684087.0
7666,2019-10-01,2479535.0,1595330.0,884205.0,905788.0,26674.0,115379.0,628606.0,1480.0,187604.0,...,535021.0,-464599.0,-293196.0,-995224.0,-100000.0,50000.0,550000.0,-107331.0,-59418.0,-621305.0
7665,2019-07-01,2609259.0,1569801.0,1039458.0,871333.0,25212.0,133349.0,795962.0,1647.0,308059.0,...,-264315.0,160630.0,-303969.0,228988.0,-150000.0,-250000.0,-250000.0,53581.0,-53750.0,61201.0


# PREPARING DATA FOR MODEL